## How to make a new Actor in FUSE

The steps below will describe how to create a new actor in FUSE. Some of the cells containing code can be run within this notebook while others will produce errors. The code that is shown is simply meant to serve as a template for the creation of actors within the actual FUSE source code. 

To begin, check under FUSE/src/actors to see if your actor fits in any of the categories already listed. Otherwise, create a folder named for the topic of your actor. Inside the existing or new folder create a file called "XYZ_actor.jl". 

Make sure to also include your actor in the main FUSE file that can be found at FUSE/src/FUSE.jl - add the following line to that file: 

In [ ]:
include(joinpath("actors", "folder_name", "XYZ_actor.jl"))

## function ActorName(dd, act; kw...)

All the building blocks of an actor can be seen in this $\textbf{ActorName}$ function (in this case, we'll name it $\textbf{ActorExample}$), which is common to all the actors. This is the function that is executed when a user calls the familiar "FUSE.ActorName(dd, act)"

Create this function in your "XYZ_actor.jl" file, replacing ActorExample with the name of your actor. 

In [ ]:
function ActorExample(dd::IMAS.dd, act::FUSE.ParametersAllActors; kw...)
    par = act.ActorExample(kw...)
    actor = ActorExample(dd, par)
    step(actor)
    finalize(actor)
    return actor
end

This function should be preceded by a docstring explaining its functionality and describing which fields in dd it stores the resulting data in. See the example below: 

In [ ]:
"""
    ActorBlanket(dd::IMAS.dd, act::ParametersAllActors; kw...)

Evaluates blankets tritium breeding ratio (TBR), heat deposition, and neutron leakage

!!! note 
    Stores data in `dd.blanket`
"""

Each line of the $\textbf{ActorExample}$ function is explained below along with the additional structures and functions that must be defined to make this function run. 

## 1. par = act.ActorExample(kw...)

The first line in the $\textbf{ActorExample}$ function refers to the parameters of the actor. These parameters define the user-selected options that are available to be toggled in your actor.  We need to create these parameters as an object so that they can be edited by a user and passed to the internal actor structure when the actor is run. 

Let's define some parameters for our example actor:

In [ ]:
Base.@kwdef mutable struct FUSEparameters__ActorExample{T<:Real} <: FUSE.ParametersActor{T}
    _parent::WeakRef = WeakRef(nothing)
    _name::Symbol = :not_set
    length::SimulationParameters.Entry{T} = SimulationParameters.Entry(T, "m", "Some description") # it's ok not to have a default, it forces users to think about what a parameter should be
    verbose::SimulationParameters.Entry{Bool} = SimulationParameters.Entry(Bool, "", "Some other decription"; default=true)
    switch::SimulationParameters.Switch{Symbol} = SimulationParameters.Switch(Symbol, [:option_a, :option_b], "", "user can only select one of these"; default=:option_a)
end

The actor parameters, like "length", "verbose" and "switch", are each of a particular type, all of which are defined in the SimulationParameters package. You can choose from $\textbf{Entries}$ (parameters that can be freely defined by the user) or $\textbf{Switches}$ (parameters that can only be chosen from a specific pre-defined set of options), and further specify the expected type (Float, Symbol, String, etc) of the $\textbf{Entry}$ or $\textbf{Switch}$ value. 

In [ ]:
using SimulationParameters

In [ ]:
my_switch = SimulationParameters.Switch{Symbol}([:option_a, :option_b], "-", "user can only select one of these"; default=:option_a)
typeof(my_switch)
my_switch.value

In [ ]:
my_switch_string = SimulationParameters.Switch{String}(["option_a", "option_b"], "-", "user can only select one of these"; default="option_a")
typeof(my_switch_string)
my_switch_string.value

In [ ]:
my_entry = SimulationParameters.Entry{Bool}("-", "a thing which can be true or false"; default = true)
my_entry.value

## 2. actor = ActorExample(dd, par)

The second line of the $\textbf{ActorExample}$ function instantiates an $\textbf{actor}$ object, which will carry both the parameters and the data structure through all the functions that make up the actor's operations. 

Let's define the object $\textbf{ActorExample}$: 

In [ ]:
mutable struct ActorExample{D,P} <: FUSE.SingleAbstractActor{D,P}
    dd::IMAS.dd{D}
    par::FUSEparameters__ActorExample{P}

    function ActorExample(dd::IMAS.dd, par::FUSEparameters__ActorExample; kw...)
        logging_actor_init(ActorExample)
        par = par(kw...)
        return new{D,P}(dd, par)
    end
    
end

The ActorExample structure shown above is defined in the first line to be a subtype of the FUSE type $\textbf{SingleAbstractActor}$. This is in contrast to being a subtype of the FUSE type $\textbf{CompoundAbstractActor}$. Does your actor run a single model or allow a user to run one or more of a set of many models? This determines whether it is $\textbf{SingleAbstractActor}$ or $\textbf{CompoundAbstractActor}$. An example of an actor that is a $\textbf{SingleAbstractActor}$ type is the CHEASE actor or the Solovev actor, while an example of a $\textbf{CompoundAbstractActor}$ is the Equilibrium actor (i.e. CHEASE and Solovev are two individual models that can be selected in the Equilibrium Actor.) 

Inside the mutable struct ActorExample is the function $\textbf{ActorExample}$. This function is an $\textit{inner constructor}$ - it defines how the ActorExample structure is initialized. 

In the constructor there's a call to the $\textbf{logging\_actor\_init}$ function - this function writes the name of the actor to the screen or a log file when the actor is run. 

The next line assigns any keyword arguments passed by the user to the actor to the act/par object - e.g. FUSE.ActorCosting(dd, act; model = :ARIES).

Finally, the $\textbf{new}$ function creates an instance of the $\textbf{ActorExample}$ object with the specific values of dd and par that have been passed to it.

## Compound actors

In the case where your actor is a $\textbf{CompoundAbstractActor}$ type, this structure definition should include an additional field, as shown below. This field carries the actor object associated to the particular model that the user has selected to run. 

In [ ]:
mutable struct ActorExample{D,P} <: FUSE.CompoundAbstractActor{D,P}
    dd::IMAS.dd{D}
    par::FUSEparameters__ActorExample{P}
    ex_actor::Union{Nothing, ActorModel1{D,P}, ActorModel2{D,P}, ActorModel3{D,P}}
end

In the compound case, the constructor for the $\textbf{ActorExample}$ object should be outside of the structure definition, and its purpose is to direct the CompoundActor to run the SingleActor or SingleActors that are specified by par.model. 

In [ ]:
function ActorExample(dd::IMAS.dd, par::FUSEparameters__ActorExample, act::FUSE.ParametersAllActors; kw...)
    logging_actor_init(ActorExample)
    par = par(kw...)
    if par.model == :Model1
        ex_actor = ActorModel1(dd, act.ActorModel1)
    elseif par.model == :Model2
        ex_actor = ActorModel2(dd, act.ActorModel2)
    elseif par.model == :Model3
        ex_actor = ActorModel3(dd, act.ActorModel3)
    else
        error("ActorExample: model = `$(par.model)` can only be one of [:Model1, :Model2, :Model3]")
    end
    return ActorExample(dd, par, act, ex_actor)
end

## 3. step(actor)

The third line in the ActorExample function is a call to $\textbf{step(actor)}$, which is the function that contains the actual actions the actor performs. 

The $\textbf{step}$ function operates on the $\textbf{actor}$ object, which carries a copy of the data structure (dd) and a copy of the act parameters. The $\textbf{step}$ function returns the $\textbf{actor}$ object after new quantities that are calculated within the step are assigned into the relevant fields in dd. 

In [ ]:
function _step(actor::ActorExample) # Assuming ActorExample is a subtype of SingleAbstractActor
    dd = actor.dd 
    par = actor.par 

    dd.example.sum_of_quantities = dd.example.quantity1 + dd.example.quantity2
    dd.example.product_of_quantities = dd.example.quantity1 * dd.example.quantity2
    
    return actor 
end

function _step(actor::ActorExample) # Assuming ActorExample is a subtype of CompoundAbstractActor
    step(actor.ex_actor)
    return actor
end

## 4. finalize(actor)

The fourth line in the $\textbf{ActorExample}$ function is a call to $\textbf{finalize(actor)}$, which is a function that may, e.g. organize the resulting data that was written to dd during the $\textbf{step}$ function for the case of a single actor, or contain a call to the $\textbf{finalize}$ of a particular model actor for the case of a compound actor. 

In [ ]:
function _finalize(actor::ActorExample) # Assuming ActorExample is a subtype of SingleAbstractActor
    sort!(dd.example.system, by=x -> x.quantity)
    return actor
end

function _finalize(actor::ActorExample) # Assuming ActorExample is a subtype of CompoundAbstractActor
    finalize(actor.ex_actor)
    return actor
end